# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 2:00PM,238681,10,SNAP15203225,"Snap Medical Industries, LLC",Released
1,Jun 2 2022 2:00PM,238681,10,SNAP6012022,"Snap Medical Industries, LLC",Released
2,Jun 2 2022 2:00PM,238681,10,SNAPA1877588,"Snap Medical Industries, LLC",Released
3,Jun 2 2022 1:53PM,238679,20,ALUR005671726,Alumier Labs Inc.,Released
4,Jun 2 2022 1:53PM,238679,20,ALUR310537413,Alumier Labs Inc.,Released
5,Jun 2 2022 1:53PM,238679,20,ALUR327952524,Alumier Labs Inc.,Released
6,Jun 2 2022 1:53PM,238679,20,ALUR360910458,Alumier Labs Inc.,Released
7,Jun 2 2022 1:53PM,238679,20,ALUR473087570,Alumier Labs Inc.,Released
8,Jun 2 2022 1:53PM,238679,20,ALUR827130700,Alumier Labs Inc.,Released
9,Jun 2 2022 1:53PM,238679,20,ALUR959456245,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,238676,Released,45
52,238677,Released,5
53,238678,Released,7
54,238679,Released,11
55,238681,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238676,NaN,NaN,45.0
238677,NaN,NaN,5.0
238678,NaN,NaN,7.0
238679,NaN,NaN,11.0
238681,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,5.0,2.0,7.0
238597,0.0,21.0,1.0
238598,0.0,0.0,2.0
238613,1.0,2.0,0.0
238616,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,5,2,7
238597,0,21,1
238598,0,0,2
238613,1,2,0
238616,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,5,2,7
1,238597,0,21,1
2,238598,0,0,2
3,238613,1,2,0
4,238616,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,5,2,7
1,238597,,21,1
2,238598,,,2
3,238613,1,2,
4,238616,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC"
3,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.
14,Jun 2 2022 1:51PM,238678,10,Emerginnova
21,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation
66,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation
71,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd"
77,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America
78,Jun 2 2022 1:33PM,238672,10,Bio-PRF
79,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.
80,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",,,3
1,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.,,,11
2,Jun 2 2022 1:51PM,238678,10,Emerginnova,,,7
3,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,,,45
4,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,,,5
5,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",,,6
6,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,,,1
7,Jun 2 2022 1:33PM,238672,10,Bio-PRF,,,1
8,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,,,1
9,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation,,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",3,,
1,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.,11,,
2,Jun 2 2022 1:51PM,238678,10,Emerginnova,7,,
3,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,45,,
4,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,5,,
5,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",6,,
6,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,1,,
7,Jun 2 2022 1:33PM,238672,10,Bio-PRF,1,,
8,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,1,,
9,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation,12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",3,,
1,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.,11,,
2,Jun 2 2022 1:51PM,238678,10,Emerginnova,7,,
3,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,45,,
4,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
1,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.,11.0,NaN,NaN
2,Jun 2 2022 1:51PM,238678,10,Emerginnova,7.0,NaN,NaN
3,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,45.0,NaN,NaN
4,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 2:00PM,238681,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
1,Jun 2 2022 1:53PM,238679,20,Alumier Labs Inc.,11.0,0.0,0.0
2,Jun 2 2022 1:51PM,238678,10,Emerginnova,7.0,0.0,0.0
3,Jun 2 2022 1:51PM,238676,10,ISDIN Corporation,45.0,0.0,0.0
4,Jun 2 2022 1:47PM,238677,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4295871,151.0,32.0,5.0
12,954561,8.0,4.0,1.0
15,1431714,13.0,36.0,20.0
16,715969,3.0,0.0,0.0
19,954626,14.0,5.0,0.0
20,1670585,40.0,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4295871,151.0,32.0,5.0
1,12,954561,8.0,4.0,1.0
2,15,1431714,13.0,36.0,20.0
3,16,715969,3.0,0.0,0.0
4,19,954626,14.0,5.0,0.0
5,20,1670585,40.0,5.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,151.0,32.0,5.0
1,12,8.0,4.0,1.0
2,15,13.0,36.0,20.0
3,16,3.0,0.0,0.0
4,19,14.0,5.0,0.0
5,20,40.0,5.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,151.0
1,12,Released,8.0
2,15,Released,13.0
3,16,Released,3.0
4,19,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,5.0,1.0,20.0,0.0,0.0,0.0
Executing,32.0,4.0,36.0,0.0,5.0,5.0
Released,151.0,8.0,13.0,3.0,14.0,40.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,5.0,1.0,20.0,0.0,0.0,0.0
1,Executing,32.0,4.0,36.0,0.0,5.0,5.0
2,Released,151.0,8.0,13.0,3.0,14.0,40.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,5.0,1.0,20.0,0.0,0.0,0.0
1,Executing,32.0,4.0,36.0,0.0,5.0,5.0
2,Released,151.0,8.0,13.0,3.0,14.0,40.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()